# Analiza 1000 družabnih iger z najvišjo oceno strani
Poglejmo si podatke o tisoč družabnih igrah z najvišjo oceno na spletni strani [BoardGameGeek](https://boardgamegeek.com/browse/boardgame).

## Prenos datotek
Uvozimo ustrezna paketa ter datoteko za analizo strani. V datoteki igre.csv so za vsako družabno igro shranjeni naslednji podatki:
- rang (mesto na seznamu, glede na oceno strani), 
- naslov,
- opis,
- leto nastanka,
- ocena strani,
- povprečna ocena,
- število glasov,

In [53]:
import pandas as pd
import numpy as np
%matplotlib inline

In [54]:
igre = pd.read_csv('igre.csv')
pd.options.display.max_rows = 20

V tabelo še dodamo en stolpec s podatkom o desetletju izida, ki nam bo pomagal pri analizi podatkov.

In [55]:
def izracunaj_desetletje(stevilo):
    return 10 * (stevilo // 10)

igre['desetletje'] = igre['leto'].apply(izracunaj_desetletje)

In [57]:
igre

,rang,ime,leto,opis,ocena_strani,povprecna_ocena,stevilo_glasov,desetletje
0,1,Brass: Birmingham,2018,"Build networks, grow industries, and navigate ...",8.414,8.59,47473,2010
1,2,Pandemic Legacy: Season 1,2015,Mutating diseases are spreading around the wor...,8.376,8.52,54001,2010
2,3,Gloomhaven,2017,Vanquish monsters with strategic cardplay. Ful...,8.346,8.58,62863,2010
3,4,Ark Nova,2021,"Plan and build a modern, scientifically manage...",8.338,8.54,45614,2020
4,5,Twilight Imperium: Fourth Edition,2017,"Build an intergalactic empire through trade, r...",8.238,8.59,24348,2010
...,...,...,...,...,...,...,...,...
995,996,Stella: Dixit Universe,2021,Players confront each other with word-images a...,6.619,7.31,3626,2020
996,997,Tragedy Looper,2011,"Players live, die, and repeat in order to outw...",6.619,7.18,4603,2010
997,998,Call to Adventure,2019,"Hero or Foe? You decide in this tile chucking,...",6.619,7.08,6408,2010
998,999,Lanterns: The Harvest Festival,2015,Decorate the lake with floating lanterns and d...,6.618,6.87,11510,2010
